In [18]:
using DuckDB
using TidierDB 

# create a new in-memory database
con = DBInterface.connect(DuckDB.DB, ":memory:")

# create a table
DBInterface.execute(con, "CREATE TABLE integers (i INTEGER)")

# insert data using a prepared statement
stmt = DBInterface.prepare(con, "INSERT INTO integers VALUES(?)")
DBInterface.execute(stmt, [42])

# query the database
results = DBInterface.execute(con, "SELECT 42 a")
print(results)

@chain db_table(con.main_connection, Symbol("information_schema.tables")) begin
    @collect
end

# DuckDB.close(con)

(a = Union{Missing, Int32}[42],)

Row,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?
1,memory,main,integers,BASE TABLE,missing,missing,missing,missing,missing,YES,NO,missing,missing


In [19]:
using DataFrames

# create a new in-memory dabase
# con = DBInterface.connect(DuckDB.DB)

# create a DataFrame
df = DataFrame(a = [1, 2, 3], b = [42, 84, 42])

# register it as a view in the database
DuckDB.register_data_frame(con, df, "my_df")

# run a SQL query over the DataFrame
results = DBInterface.execute(con, "SELECT * FROM my_df")
print(results)

@chain db_table(con.main_connection, Symbol("information_schema.tables")) begin
    @collect
end

(a = Union{Missing, Int64}[1, 2, 3], b = Union{Missing, Int64}[42, 84, 42])

Row,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?
1,memory,main,integers,BASE TABLE,missing,missing,missing,missing,missing,YES,NO,missing,missing
2,memory,main,my_df,VIEW,missing,missing,missing,missing,missing,NO,NO,missing,missing


In [20]:
using DuckDB, DataFrames, Dates

db = DuckDB.DB()
# create a table
DBInterface.execute(db,
    "CREATE OR REPLACE TABLE data(id INTEGER PRIMARY KEY, value FLOAT, timestamp TIMESTAMP, date DATE)")
# create data to insert
len = 100
df = DataFrames.DataFrame(
        id = collect(1:len),
        value = rand(len),
        timestamp = Dates.now() + Dates.Second.(1:len),
        date = Dates.today() + Dates.Day.(1:len)
    )
# append data by row
appender = DuckDB.Appender(db, "data")
for i in eachrow(df)
    for j in i
        DuckDB.append(appender, j)
    end
    DuckDB.end_row(appender)
end


# flush the appender after all rows
DuckDB.flush(appender)

results = DBInterface.execute(db, "SELECT * FROM data")
print(results)


# db_1 = TidierDB.connect(:duckdb);

@chain db_table(db.main_connection, Symbol("information_schema.tables")) begin
    @collect
end

# DuckDB.close(appender)

(id = Union{Missing, Int32}[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], value = Union{Missing, Float32}[0.5115417f0, 0.708908f0, 0.021334026f0, 0.85995907f0, 0.3196257f0, 0.91510665f0, 0.9875123f0, 0.5771857f0, 0.14563368f0, 0.4462561f0, 0.21808983f0, 0.4627787f0, 0.53883165f0, 0.6836705f0, 0.386917f0, 0.7242834f0, 0.34874564f0, 0.36838245f0, 0.744039f0, 0.21246241f0, 0.83622915f0, 0.029603733f0, 0.39501277f0, 0.4657131f0, 0.012434994f0, 0.7331246f0, 0.72550625f0, 0.09283919f0, 0.5297053f0, 0.026309423f0, 0.26267546f0, 0.4400793f0, 0.69617844f0, 0.9120795f0, 0.13488404f0, 0.008660946f0, 0.5175384f0, 0.26788625f0, 0.74965715f0, 0.49092415f0, 0.

Row,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?
1,memory,main,data,BASE TABLE,missing,missing,missing,missing,missing,YES,NO,missing,missing
